Before we start, we are importing all of the necessary libraries.

In [1]:
# importing libraries
import pandas as pd # to read csv file
import numpy as np # for matrix operations

!conda install -c conda-forge folium=0.5.0 --yes
import folium # to draw map

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # to get latitude and longitude

import requests # to request
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans # for clustering

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    numpy-1.18.1               |   py36h95a1406_0         5.2 MB  conda-forge
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    scipy-1.4.1                |   py36h921218d_0        18.9 MB  conda-forge


At the following junk we are reading the latitude and longitude values of the Bursa's boroughs. I have created this values by calling geolocator library for each borough. Then i recorded values into an Excel sheet and save it as csv to share it with you too. Unfortunately, i did not find the neighborhood values for boroughs. So i had to assign borough values for neighborhood values too.

In [2]:
bursa_data = pd.read_csv("Bursa_Data.csv", sep = ";")
bursa_data

,City,Borough,Neighborhood,Latitude,Longitude
0,Bursa,Büyükorhan,Büyükorhan,"39,7441994","28,87014551"
1,Bursa,Gemlik,Gemlik,"40,4301652","29,1570698"
2,Bursa,Gürsu,Gürsu,"40,26436655","29,20695247"
3,Bursa,Harmancık,Harmancık,"39,69986915","29,15177738"
4,Bursa,İnegöl,İnegöl,"40,0800144","29,5096448"
5,Bursa,İznik,İznik,"40,4303445","29,7223732"
6,Bursa,Karacabey,Karacabey,"40,2375461","28,39600197"
7,Bursa,Keles,Keles,"39,8898902","29,27175259"
8,Bursa,Kestel,Kestel,"40,186927","29,30473163"
9,Bursa,Mudanya,Mudanya,"40,3254848","28,76945331"


As you can see above we have a problem. Because of the Excel settings latitude and longitude values for boroughs is separated via comma and again because of the Excel settings both of the latitude and longitude values are 'string' instead of 'float'. So at the following junk first i am replacing ',' with '.' and change the values as 'numeric'

In [3]:
bursa_data['Longitude'] = bursa_data['Longitude'].str.replace(',','.')
bursa_data['Latitude'] = bursa_data['Latitude'].str.replace(',','.')
bursa_data[["Latitude", "Longitude"]] = bursa_data[["Latitude", "Longitude"]].apply(pd.to_numeric)

So let's check both of the values and types. 'dtypes' is a method which shows you the data type of the columns and data frame.

In [4]:
bursa_data.dtypes

City             object
Borough          object
Neighborhood     object
Latitude        float64
Longitude       float64
dtype: object

In [5]:
bursa_data

,City,Borough,Neighborhood,Latitude,Longitude
0,Bursa,Büyükorhan,Büyükorhan,39.744199,28.870146
1,Bursa,Gemlik,Gemlik,40.430165,29.157070
2,Bursa,Gürsu,Gürsu,40.264367,29.206952
3,Bursa,Harmancık,Harmancık,39.699869,29.151777
4,Bursa,İnegöl,İnegöl,40.080014,29.509645
5,Bursa,İznik,İznik,40.430345,29.722373
6,Bursa,Karacabey,Karacabey,40.237546,28.396002
7,Bursa,Keles,Keles,39.889890,29.271753
8,Bursa,Kestel,Kestel,40.186927,29.304732
9,Bursa,Mudanya,Mudanya,40.325485,28.769453


Before we create a map we need the latitude and longitude values of Bursa. At the following junk we are using geolocator to find latitude and longitudes of the Bursa.

In [6]:
address = 'Bursa'

geolocator = Nominatim(user_agent="bursa_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bursa are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bursa are 40.182737, 29.0678352.


Now we are creating a map of Bursa which includes the Borough values.

In [7]:
# create map of Toronto using latitude and longitude values
map_bursa = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(bursa_data['Latitude'], bursa_data['Longitude'], bursa_data['Borough'], bursa_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bursa)  
    
map_bursa

And now let's initial our client_id and client_secret at Foursquare API. I have deleted my id and secret. So if you wish to run the notebook you need to use your own client id and secret.

In [ ]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

At this junk i have used the getNearbyVenues function from the lab exercises. I have assigned 4180 meters as radius value. Because the smallest borough of the Bursa is 'Yıldırım' which is 110 kilometers square. So if we try to draw a circle from the coordinate center of the 'Yıldırım', radius of the circle will be approximately 4.180 kilometers.  

In [9]:
def getNearbyVenues(names, latitudes, longitudes, radius=4180):
    LIMIT = 500
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now let's call the function.

In [10]:
borough_venues = getNearbyVenues(names=bursa_data['Neighborhood'],
                                   latitudes=bursa_data['Latitude'],
                                   longitudes=bursa_data['Longitude']
                                  )

Büyükorhan
Gemlik
Gürsu
Harmancık
İnegöl
İznik
Karacabey
Keles
Kestel
Mudanya
Mustafakemalpaşa
Nilüfer
Orhaneli
Orhangazi
Osmangazi
Yenişehir
Yıldırım


Let's check the shape and first and last five values of the data frame.

In [11]:
print(borough_venues.shape)
borough_venues.head()

(904, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Büyükorhan,39.744199,28.870146,Ekomini,39.771730,28.886716,Convenience Store
1,Büyükorhan,39.744199,28.870146,Büyükorhan Yıldırım Belediyesi Parkı,39.775058,28.885938,Theme Park
2,Büyükorhan,39.744199,28.870146,Kapalı Pazar,39.769340,28.886484,Market
3,Büyükorhan,39.744199,28.870146,Büyükorhan Teras Cafe,39.770685,28.886360,Mediterranean Restaurant
4,Büyükorhan,39.744199,28.870146,Büyükorhan Dag Yolu,39.774393,28.868063,Forest


In [12]:
borough_venues.tail()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
899,Yıldırım,40.190099,29.097653,Setbaşı Roma Dondurma,40.180583,29.070860,Ice Cream Shop
900,Yıldırım,40.190099,29.097653,Tuz Pazarı Çarşısı,40.185066,29.065252,Farmers Market
901,Yıldırım,40.190099,29.097653,Ulu Cami,40.183965,29.062331,Mosque
902,Yıldırım,40.190099,29.097653,Bağdat Hurma Tatlıcısı,40.194273,29.064308,Dessert Shop
903,Yıldırım,40.190099,29.097653,Meşhur Kayhan Pideli Köfte,40.184169,29.069145,Turkish Restaurant


We have 904 venues for Bursa's boroughs. At the following junk i have grouped the boroughs according to the neighborhood. At the very start of the notebook we assumed that 'Neighborhood value is the borough value' so we will analyze the venues according to boroughs in other words.

In [13]:
borough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Büyükorhan,10,10,10,10,10,10
Gemlik,100,100,100,100,100,100
Gürsu,33,33,33,33,33,33
Harmancık,17,17,17,17,17,17
Karacabey,95,95,95,95,95,95
Keles,17,17,17,17,17,17
Kestel,28,28,28,28,28,28
Mudanya,18,18,18,18,18,18
Mustafakemalpaşa,12,12,12,12,12,12


And check the unique categories of the venues.

In [14]:
print('There are {} uniques categories.'.format(len(borough_venues['Venue Category'].unique())))

There are 180 uniques categories.


Now we have an encoding operation for the categories.

In [14]:
# one hot encoding
bursa_onehot_encode = pd.get_dummies(borough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bursa_onehot_encode['Neighborhood'] = borough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bursa_onehot_encode.columns[-1]] + list(bursa_onehot_encode.columns[:-1])
bursa_onehot_encode = bursa_onehot_encode[fixed_columns]

bursa_onehot_encode.head()

,Neighborhood,Accessories Store,Airport Terminal,Arcade,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Athletics & Sports,Auto Garage,BBQ Joint,...,Travel & Transport,Tree,Tunnel,Turkish Coffeehouse,Turkish Home Cooking Restaurant,Turkish Restaurant,Vietnamese Restaurant,Water Park,Waterfront,Wings Joint
0,Büyükorhan,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Büyükorhan,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Büyükorhan,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Büyükorhan,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Büyükorhan,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
bursa_onehot_encode.tail()

,Neighborhood,Accessories Store,Airport Terminal,Arcade,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Athletics & Sports,Auto Garage,BBQ Joint,...,Travel & Transport,Tree,Tunnel,Turkish Coffeehouse,Turkish Home Cooking Restaurant,Turkish Restaurant,Vietnamese Restaurant,Water Park,Waterfront,Wings Joint
899,Yıldırım,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
900,Yıldırım,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
901,Yıldırım,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
902,Yıldırım,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
903,Yıldırım,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


Let's check the shape of the data frame.

In [16]:
bursa_onehot_encode.shape

(904, 181)

At this junk we are grouping the Neighborhood column of the bursa_onehot_encode and finding the mean of the values.

In [17]:
bursa_grouped_data = bursa_onehot_encode.groupby('Neighborhood').mean().reset_index()
bursa_grouped_data

,Neighborhood,Accessories Store,Airport Terminal,Arcade,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Athletics & Sports,Auto Garage,BBQ Joint,...,Travel & Transport,Tree,Tunnel,Turkish Coffeehouse,Turkish Home Cooking Restaurant,Turkish Restaurant,Vietnamese Restaurant,Water Park,Waterfront,Wings Joint
0,Büyükorhan,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Gemlik,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.000000,0.020000,0.010000
2,Gürsu,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030303,0.000000,0.030303,...,0.000000,0.060606,0.030303,0.030303,0.000000,0.030303,0.000000,0.000000,0.000000,0.000000
3,Harmancık,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.058824,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000
4,Karacabey,0.000000,0.000000,0.031579,0.000000,0.000000,0.010526,0.000000,0.010526,0.000000,...,0.000000,0.000000,0.000000,0.010526,0.000000,0.010526,0.010526,0.010526,0.000000,0.000000
5,Keles,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000
6,Kestel,0.035714,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.035714,0.000000,0.035714,0.035714,0.000000,0.000000,0.000000,0.000000,0.000000
7,Mudanya,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.055556,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Mustafakemalpaşa,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Nilüfer,0.000000,0.000000,0.011111,0.000000,0.000000,0.000000,0.022222,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000


The shape of the bursa_grouped_data data frame is 17x181. This 17 represents number the boroughs and the 181 represents the borough name and 180 unique venues.

In [19]:
bursa_grouped_data.shape

(17, 181)

We are using return_most_common_venues function from the lab exercises to get top venues and using the function for the top 10 venues. 

In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [23]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = bursa_grouped_data['Neighborhood']

for ind in np.arange(bursa_grouped_data.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bursa_grouped_data.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Büyükorhan,Scenic Lookout,Food Truck,Forest,Convenience Store,Mediterranean Restaurant,Market,Theme Park,Café,Mountain,Fish & Chips Shop
1,Gemlik,Café,Seafood Restaurant,Convenience Store,Gym / Fitness Center,Coffee Shop,Turkish Restaurant,Dessert Shop,Restaurant,Burger Joint,Breakfast Spot
2,Gürsu,Mountain,Forest,Tree,Tea Room,Plaza,Kafenio,Convenience Store,Moving Target,Fast Food Restaurant,Soccer Stadium
3,Harmancık,Café,Campground,Mountain,Forest,Business Service,Soccer Stadium,Bus Station,Cafeteria,Tunnel,Park
4,Karacabey,Café,Convenience Store,Kofte Place,Arcade,Fast Food Restaurant,Soup Place,Kebab Restaurant,Nightclub,Cigkofte Place,Gym / Fitness Center


Now we are going to set the clusters. I have used 5 cluster for this project. 

In [24]:
# set number of clusters
kclusters = 5

bursa_grouped_clustering = bursa_grouped_data.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bursa_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 0, 2, 0, 0, 0, 0, 0, 1, 0], dtype=int32)

Now we are going to add the labels of clusters to the neighborhoods_venues_sorted data frame and merge it with the toronto_central data frame.    

In [28]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

bursa_data_merged = bursa_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
bursa_data_merged = bursa_data_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

bursa_data_merged.head() # check the last columns!

,City,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bursa,Büyükorhan,Büyükorhan,39.744199,28.870146,3,Scenic Lookout,Food Truck,Forest,Convenience Store,Mediterranean Restaurant,Market,Theme Park,Café,Mountain,Fish & Chips Shop
1,Bursa,Gemlik,Gemlik,40.430165,29.157070,0,Café,Seafood Restaurant,Convenience Store,Gym / Fitness Center,Coffee Shop,Turkish Restaurant,Dessert Shop,Restaurant,Burger Joint,Breakfast Spot
2,Bursa,Gürsu,Gürsu,40.264367,29.206952,2,Mountain,Forest,Tree,Tea Room,Plaza,Kafenio,Convenience Store,Moving Target,Fast Food Restaurant,Soccer Stadium
3,Bursa,Harmancık,Harmancık,39.699869,29.151777,0,Café,Campground,Mountain,Forest,Business Service,Soccer Stadium,Bus Station,Cafeteria,Tunnel,Park
4,Bursa,İnegöl,İnegöl,40.080014,29.509645,0,Café,Restaurant,Kofte Place,Gym / Fitness Center,Steakhouse,Hotel,Arcade,Motorcycle Shop,Soup Place,Bagel Shop


Now we can draw the map which includes our clusters.

In [29]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bursa_data_merged['Latitude'], bursa_data_merged['Longitude'], bursa_data_merged['Neighborhood'], bursa_data_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

At the end let's check our 5 clusters to find out the most common venues for each cluster.

In [31]:
bursa_data_merged.loc[bursa_data_merged['Cluster Labels'] == 0, bursa_data_merged.columns[[1] + list(range(5, bursa_data_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Gemlik,0,Café,Seafood Restaurant,Convenience Store,Gym / Fitness Center,Coffee Shop,Turkish Restaurant,Dessert Shop,Restaurant,Burger Joint,Breakfast Spot
3,Harmancık,0,Café,Campground,Mountain,Forest,Business Service,Soccer Stadium,Bus Station,Cafeteria,Tunnel,Park
4,İnegöl,0,Café,Restaurant,Kofte Place,Gym / Fitness Center,Steakhouse,Hotel,Arcade,Motorcycle Shop,Soup Place,Bagel Shop
5,İznik,0,Café,Hotel,Art Gallery,Beach,Castle,Restaurant,Resort,Scenic Lookout,Kebab Restaurant,Turkish Restaurant
6,Karacabey,0,Café,Convenience Store,Kofte Place,Arcade,Fast Food Restaurant,Soup Place,Kebab Restaurant,Nightclub,Cigkofte Place,Gym / Fitness Center
7,Keles,0,Convenience Store,Steakhouse,Tea Room,Pub,Mountain,Bistro,Bed & Breakfast,Café,Restaurant,Clothing Store
8,Kestel,0,Beach,Breakfast Spot,Department Store,Lake,Kofte Place,Intersection,Garden,Rest Area,Moving Target,Knitting Store
9,Mudanya,0,Farm,Café,Lake,Forest,Beer Garden,Beach,Campground,BBQ Joint,Park,Soccer Stadium
11,Nilüfer,0,Café,Steakhouse,Buffet,Farm,Coffee Shop,Pide Place,Turkish Restaurant,Bakery,Restaurant,Soccer Field
14,Osmangazi,0,Historic Site,Café,Bakery,Seafood Restaurant,Turkish Restaurant,Cosmetics Shop,Mosque,Park,History Museum,Gym


In [32]:
bursa_data_merged.loc[bursa_data_merged['Cluster Labels'] == 1, bursa_data_merged.columns[[1] + list(range(5, bursa_data_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Mustafakemalpaşa,1,Construction & Landscaping,Turkish Coffeehouse,River,Plaza,Café,Indian Chinese Restaurant,Pier,Forest,Nature Preserve,Flea Market


In [33]:
bursa_data_merged.loc[bursa_data_merged['Cluster Labels'] == 2, bursa_data_merged.columns[[1] + list(range(5, bursa_data_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Gürsu,2,Mountain,Forest,Tree,Tea Room,Plaza,Kafenio,Convenience Store,Moving Target,Fast Food Restaurant,Soccer Stadium
12,Orhaneli,2,Mountain,Hotel,Spa,River,Nature Preserve,Campground,Farm,Wings Joint,Farmers Market,Flower Shop


In [34]:
bursa_data_merged.loc[bursa_data_merged['Cluster Labels'] == 3, bursa_data_merged.columns[[1] + list(range(5, bursa_data_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Büyükorhan,3,Scenic Lookout,Food Truck,Forest,Convenience Store,Mediterranean Restaurant,Market,Theme Park,Café,Mountain,Fish & Chips Shop


In [35]:
bursa_data_merged.loc[bursa_data_merged['Cluster Labels'] == 4, bursa_data_merged.columns[[1] + list(range(5, bursa_data_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Orhangazi,4,Breakfast Spot,Recreation Center,Athletics & Sports,Restaurant,Lake,Farm,Seafood Restaurant,Trail,Nature Preserve,Park


Thank you for your patience and for your feedbacks. Now you know the 'What' to open in 'Where' question for Bursa :)